### **Definir el directorio de trabajo con Google Colab**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.

In [1]:
import os
from google.colab import drive 
# Enlazar a la carpeta data de la carpeta compartida en Drive
drive.mount('/content/drive')
working_dir = '/content/drive/MyDrive/DS4A-Team12/data'
os.chdir(working_dir)
!pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1dkrMUgtvHu8Va_rPx6L6g6q-151MIsTK/DS4A-Team12/data


In [2]:
# Librerias relevantes 
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.api         as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multitest as smm
#import os
from scipy import stats, special
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
from sklearn import neighbors
from sklearn import ensemble
%matplotlib inline
plt.style.use('ggplot')

# Opciones de visualización
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **Abrir base de datos unidos**
Abre el archivo `Sociodemo_pre.parquet` (5945349 registros, 148 MB) como un dataframe de nombre `soc`, con toda la base de datos, preprocesada para asignarle a cada variable el tipo de dato correspondiente.

Abre también el diccionario del archivo `sociodemo_datatypes_low_memo.json` que permite asignar a las variables tipos de datos para un uso más eficiente de la memoria (641 MB).

**Correr Localmente**

In [ ]:
# import json
# # Abre el diccionario con la estructura de datos definida|
# soc_datatypes_file = open('DATA/sociodemo_datatypes_low_memo.json', 'r')
# soc_datatypes = json.load(soc_datatypes_file)
# # Abre el dataframe
# df = pd.read_parquet('DATA/join_sociodemo_tomasDPI_tagged.parquet')
# df = df.astype(soc_datatypes)

In [ ]:
import json
# Abre el diccionario con la estructura de datos definida|
soc_datatypes_file = open('datatypes_dictionaries/sociodemo_datatypes_low_memo.json', 'r')
soc_datatypes = json.load(soc_datatypes_file)
# Abre el dataframe
df = pd.read_parquet('preprocessed_data/join_sociodemo_tomasDPI_tagged.parquet')
df = df.astype(soc_datatypes)

In [3]:
df2 = pd.read_parquet('preprocessed_data/tomas_max_500.parquet')

In [4]:
df2

,IdToma,Registro,Vigencia,Toma,Servicio,FechaValoracionNutricional,EdadMeses,FechaMedicionPerimetroBraquial,MedicionPerimetroBraquial,Peso,Talla,ZScoreTallaEdad,ZScorePesoEdad,ZScorePesoTalla,ZScoreIMC,EstadoTallaEdad,EstadoPesoEdad,EstadoPesoTalla,EstadoIMC,Flag,FechaRegistroSaludNutricion,PresentaCarneVacunacion,ControlesCrecimDesarrollo,AntecedentePremadurez,Direccion,IdBeneficiario,Id,FechaNacimiento,Sexo,Estrato,cod_mpio,cod_dpto,ingresos_promP_imp
0,58,988331,2017,1,CDI SIN ARRIENDO - INSTITUCIONAL INTEGRAL,2017-01-19,45.0,NaT,NaN,17.00,100.000000,-0.47,0.51,1.21,1.21,Talla adecuada para la edad,Peso adecuado para la edad,Riesgo de sobrepeso,Riesgo de sobrepeso,0,NaT,S,1,NaN,Primera Infancia,82480,3041941,2013-04-03,1.0,2,68533,68,666666.0
1,59,988331,2017,2,CDI SIN ARRIENDO - INSTITUCIONAL INTEGRAL,2017-05-15,49.0,NaT,NaN,17.90,102.000000,-0.49,0.59,1.37,1.38,Talla adecuada para la edad,Peso adecuado para la edad,Riesgo de sobrepeso,Riesgo de sobrepeso,0,NaT,S,1,NaN,Primera Infancia,82480,3041941,2013-04-03,1.0,2,68533,68,666666.0
2,60,988331,2017,3,CDI SIN ARRIENDO - INSTITUCIONAL INTEGRAL,2017-08-11,52.0,NaT,NaN,18.60,105.000000,-0.16,0.66,1.15,1.16,Talla adecuada para la edad,Peso adecuado para la edad,Riesgo de sobrepeso,Riesgo de sobrepeso,0,NaT,S,1,NaN,Primera Infancia,82480,3041941,2013-04-03,1.0,2,68533,68,666666.0
3,61,988331,2017,4,CDI SIN ARRIENDO - INSTITUCIONAL INTEGRAL,2017-10-13,54.0,NaT,NaN,18.60,106.000000,-0.19,0.50,0.93,0.95,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,1,NaN,Primera Infancia,82480,3041941,2013-04-03,1.0,2,68533,68,666666.0
4,79,988334,2017,1,CDI SIN ARRIENDO - INSTITUCIONAL INTEGRAL,2017-02-01,22.0,NaT,NaN,10.71,77.800003,-3.04,-0.96,0.76,1.41,Retraso en talla,Peso adecuado para la edad,Peso adecuado para la talla,Riesgo de sobrepeso,0,NaT,S,0,NaN,Primera Infancia,118954,1181650,2016-03-13,1.0,1,20011,20,61000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2614030,19271179,4697547,2018,1N,ESTRATEGIAS DE DESARROLLO ALIMENTARIO O NUTRIC...,2019-12-02,8.0,2019-12-02,13.0,6.00,68.000000,-0.78,-1.53,-1.47,-1.00,Talla adecuada para la edad,Riesgo de peso bajo para la edad,Riesgo de desnutrición aguda,Riesgo para la delgadez,0,2019-12-20 10:05:40,S,0,N,Nutrición,18971873,809528,2019-03-09,1.0,1,13688,13,306339.0
2614031,19271181,4697548,2018,1N,ESTRATEGIAS DE DESARROLLO ALIMENTARIO O NUTRIC...,2019-12-02,36.0,2019-12-02,13.0,11.00,89.000000,-1.92,-2.22,-1.78,-1.00,Riesgo de baja talla,Desnutrición global,Riesgo de desnutrición aguda,Riesgo para la delgadez,0,2019-12-19 22:34:05,S,0,NaN,Nutrición,18971933,4682168,2016-11-29,0.0,0,13688,13,123336.0
2614032,19271223,4699453,2018,1N,ESTRATEGIAS DE DESARROLLO ALIMENTARIO O NUTRIC...,2019-12-10,34.0,2019-12-10,13.0,10.00,86.000000,-2.04,-2.11,-1.31,-1.00,Retraso en talla,Desnutrición global,Riesgo de desnutrición aguda,Riesgo para la delgadez,0,2019-12-20 21:23:39,S,1,NaN,Nutrición,18972188,3988754,2017-01-17,0.0,1,94001,94,176666.0
2614033,19271261,4699472,2018,1N,ESTRATEGIAS DE DESARROLLO ALIMENTARIO O NUTRIC...,2019-12-10,12.0,2019-12-10,12.0,8.00,72.000000,-1.62,-1.82,-1.43,-1.00,Riesgo de baja talla,Riesgo de peso bajo para la edad,Riesgo de desnutrición aguda,Riesgo para la delgadez,0,2019-12-21 08:38:51,S,1,N,Nutrición,18972502,3990490,2018-11-23,1.0,1,94001,94,233333.0


In [ ]:
df.tail()

,Cod_clase,Ind_grupo_sisben_4,Ind_nivel_sisben_4,Tip_vivienda,Ind_tiene_energia,Ind_tiene_alcantarillado,Ind_tiene_gas,Ind_tiene_recoleccion,Ind_tiene_acueducto,Num_hogares_vivienda,Tip_ocupa_vivienda,Ind_tiene_cocina,Ind_tiene_nevera,Ind_tiene_lavadora,Ind_tiene_pc,Ind_tiene_internet,Num_personas_hogar,Sexo,Ind_discap_ninguna,Tip_seg_social,Ind_acudio_salud,Ind_fue_atendido_salud,Tip_cuidado_niños,Ind_recibe_comida,Ind_leer_escribir,Ind_estudia,privado_logroeduca,privado_analfabe,privado_Inasistencia,privado_Rezago,privado_primera_inf,privado_trabajo_inf,privado_desemplarga,privado_trabajoinfo,privado_asegura,privado_accesosalud,privado_agua,privado_excreta,privado_suelo,privado_pared,privado_hacina,Noprivaciones,IndicadorPobrezaMulti,ingresos_promP_imp,gasto_ppers_imp,gasto_alim_ppers_imp,porc_gasto_alim,union_temp,n_ninos,Tip_hijo,Jefat_fem,Tip_familia,Gene_Ingr,Nivel_padreUni,Nivel_madreUni,anno_encuesta,Uni_dias_agua,Ind_afec_eventonatural,num_afec_eventonatural,Edad_padres_mayor,Edad_padres_menor,ComparteCocina_Sanitario,Estrato,cod_mpio,cod_dpto,FechaNacimiento,IdBeneficiario,Id,desnutricion,recuperacion,reincidencia
990768,3,C,17,1,0,0,0,0,0,1,4,1,0,0,0,0,3,0.0,1.0,0,1.0,1.0,3,1,9,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,5.85,1,345666.0,236666.0,133333.0,56.0,0,1,2,0,4,4,2,2,2017,0.0,0,0,36,34,1,0,99524,99,2014-11-27,11419702,5945432,False,False,False
990769,3,C,30,2,1,0,0,0,0,1,2,1,0,0,0,0,4,1.0,1.0,3,9.0,9.0,1,1,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,700000.0,283750.0,70000.0,24.0,0,1,2,1,4,1,8,4,2019,0.0,1,1,30,30,1,1,99001,99,2016-07-18,15322643,5945442,False,False,False
990770,3,D,36,1,0,0,0,0,0,2,4,1,0,0,0,0,3,0.0,1.0,1,9.0,9.0,2,1,9,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,2.70,0,266666.0,356666.0,150000.0,42.0,0,1,2,0,6,4,6,5,2018,0.0,1,6,33,28,1,0,99624,99,2016-05-01,17388731,5945444,False,False,False
990771,3,D,51,1,1,0,0,0,0,1,3,1,0,0,0,0,3,1.0,1.0,3,9.0,9.0,1,1,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,400000.0,113333.0,100000.0,88.0,0,1,2,0,6,3,6,5,2019,0.0,0,0,27,23,0,1,99001,99,2015-09-24,13495606,5945448,False,False,False
990772,3,D,31,1,0,0,0,0,0,1,3,1,0,0,0,0,3,1.0,1.0,3,9.0,9.0,5,1,9,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.50,0,466666.0,166666.0,166666.0,100.0,0,1,2,0,6,3,4,4,2019,0.0,0,0,27,24,1,0,99001,99,2018-02-02,16919055,5945451,False,False,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990773 entries, 0 to 990772
Data columns (total 71 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Cod_clase                 990773 non-null  category      
 1   Ind_grupo_sisben_4        990773 non-null  category      
 2   Ind_nivel_sisben_4        990773 non-null  UInt8         
 3   Tip_vivienda              990773 non-null  category      
 4   Ind_tiene_energia         990773 non-null  category      
 5   Ind_tiene_alcantarillado  990773 non-null  category      
 6   Ind_tiene_gas             990773 non-null  category      
 7   Ind_tiene_recoleccion     990773 non-null  category      
 8   Ind_tiene_acueducto       990773 non-null  category      
 9   Num_hogares_vivienda      990773 non-null  UInt8         
 10  Tip_ocupa_vivienda        990773 non-null  category      
 11  Ind_tiene_cocina          990773 non-null  category      
 12  In

In [ ]:
#train[df.columns[3:47]].drop(columns=['union_temp'])

In [ ]:
df.head()

,Cod_clase,Ind_grupo_sisben_4,Ind_nivel_sisben_4,Tip_vivienda,Ind_tiene_energia,Ind_tiene_alcantarillado,Ind_tiene_gas,Ind_tiene_recoleccion,Ind_tiene_acueducto,Num_hogares_vivienda,Tip_ocupa_vivienda,Ind_tiene_cocina,Ind_tiene_nevera,Ind_tiene_lavadora,Ind_tiene_pc,Ind_tiene_internet,Num_personas_hogar,Sexo,Ind_discap_ninguna,Tip_seg_social,Ind_acudio_salud,Ind_fue_atendido_salud,Tip_cuidado_niños,Ind_recibe_comida,Ind_leer_escribir,Ind_estudia,privado_logroeduca,privado_analfabe,privado_Inasistencia,privado_Rezago,privado_primera_inf,privado_trabajo_inf,privado_desemplarga,privado_trabajoinfo,privado_asegura,privado_accesosalud,privado_agua,privado_excreta,privado_suelo,privado_pared,privado_hacina,Noprivaciones,IndicadorPobrezaMulti,ingresos_promP_imp,gasto_ppers_imp,gasto_alim_ppers_imp,porc_gasto_alim,union_temp,n_ninos,Tip_hijo,Jefat_fem,Tip_familia,Gene_Ingr,Nivel_padreUni,Nivel_madreUni,anno_encuesta,Uni_dias_agua,Ind_afec_eventonatural,num_afec_eventonatural,Edad_padres_mayor,Edad_padres_menor,ComparteCocina_Sanitario,Estrato,cod_mpio,cod_dpto,FechaNacimiento,IdBeneficiario,Id,desnutricion,recuperacion,reincidencia
0,1,A,5,2,1,1,1,1,1,2,1,1,0,0,0,0,3,0.0,1.0,3,1.0,1.0,2,1,9,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2.10,0,66666.0,240000.0,100000.0,41.0,0,2,3,1,4,1,8,4,2019,7.0,0,0,30,30,1,3,11001,11,2017-12-26,16571555,3,False,False,False
1,1,A,4,2,1,1,0,1,1,1,1,1,0,0,0,0,5,0.0,1.0,3,9.0,9.0,1,1,9,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3.60,0,120000.0,156000.0,56000.0,35.0,0,4,3,1,1,2,8,3,2019,7.0,0,0,30,30,1,2,11001,11,2015-09-29,15753361,10,False,False,False
2,1,A,4,2,1,1,0,1,1,1,1,1,0,0,0,0,5,0.0,1.0,3,9.0,9.0,9,9,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3.60,0,120000.0,156000.0,56000.0,35.0,0,4,3,1,1,2,8,3,2019,7.0,0,0,30,30,1,2,11001,11,2013-12-03,9899861,14,False,False,False
3,1,A,1,2,1,1,1,1,1,1,1,1,1,0,0,0,3,0.0,1.0,0,9.0,9.0,2,1,9,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,4.35,0,30000.0,326666.0,100000.0,30.0,0,1,2,1,6,1,3,3,2019,7.0,0,0,33,23,0,2,11001,11,2014-06-28,14146154,23,False,False,False
4,1,A,5,1,1,1,1,1,1,2,1,1,0,0,0,0,4,1.0,0.0,3,9.0,9.0,9,9,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,4.35,0,150000.0,247500.0,75000.0,30.0,0,2,3,0,6,3,2,2,2019,7.0,0,0,47,31,0,2,11001,11,2011-12-27,11313127,33,False,False,False


In [ ]:
# conviertiendo en float
for col in df.columns[3:65]:
  df[col] = df[col].astype('float')

df1 = df.drop(columns = ['desnutricion', 'recuperacion', 'union_temp','FechaNacimiento'])

#bool_val = True
#df1['reincidencia'] = (df1['reincidencia'] == 'TRUE').astype(int)
#df1['reincidencia'] = (df1['reincidencia'] == 'FALSE').astype(int)

#df1.reincidencia = int(df1.reincidencia == True)

#df1.reinicidencia = df1.reincidencia.replace(int({"TRUE" : 1,"FALSE" : 0}))

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990773 entries, 0 to 990772
Data columns (total 67 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Cod_clase                 990773 non-null  category
 1   Ind_grupo_sisben_4        990773 non-null  category
 2   Ind_nivel_sisben_4        990773 non-null  UInt8   
 3   Tip_vivienda              990773 non-null  float64 
 4   Ind_tiene_energia         990773 non-null  float64 
 5   Ind_tiene_alcantarillado  990773 non-null  float64 
 6   Ind_tiene_gas             990773 non-null  float64 
 7   Ind_tiene_recoleccion     990773 non-null  float64 
 8   Ind_tiene_acueducto       990773 non-null  float64 
 9   Num_hogares_vivienda      990773 non-null  float64 
 10  Tip_ocupa_vivienda        990773 non-null  float64 
 11  Ind_tiene_cocina          990773 non-null  float64 
 12  Ind_tiene_nevera          990773 non-null  float64 
 13  Ind_tiene_lavadora        990

In [ ]:
df1['Ind_grupo_sisben_4'] = pd.get_dummies(df1['Ind_grupo_sisben_4'])

ValueError: Columns must be same length as key

In [ ]:
df1

,Cod_clase,Ind_grupo_sisben_4,Ind_nivel_sisben_4,Tip_vivienda,Ind_tiene_energia,Ind_tiene_alcantarillado,Ind_tiene_gas,Ind_tiene_recoleccion,Ind_tiene_acueducto,Num_hogares_vivienda,Tip_ocupa_vivienda,Ind_tiene_cocina,Ind_tiene_nevera,Ind_tiene_lavadora,Ind_tiene_pc,Ind_tiene_internet,Num_personas_hogar,Sexo,Ind_discap_ninguna,Tip_seg_social,Ind_acudio_salud,Ind_fue_atendido_salud,Tip_cuidado_niños,Ind_recibe_comida,Ind_leer_escribir,Ind_estudia,privado_logroeduca,privado_analfabe,privado_Inasistencia,privado_Rezago,privado_primera_inf,privado_trabajo_inf,privado_desemplarga,privado_trabajoinfo,privado_asegura,privado_accesosalud,privado_agua,privado_excreta,privado_suelo,privado_pared,privado_hacina,Noprivaciones,IndicadorPobrezaMulti,ingresos_promP_imp,gasto_ppers_imp,gasto_alim_ppers_imp,porc_gasto_alim,n_ninos,Tip_hijo,Jefat_fem,Tip_familia,Gene_Ingr,Nivel_padreUni,Nivel_madreUni,anno_encuesta,Uni_dias_agua,Ind_afec_eventonatural,num_afec_eventonatural,Edad_padres_mayor,Edad_padres_menor,ComparteCocina_Sanitario,Estrato,cod_mpio,cod_dpto,IdBeneficiario,Id,reincidencia
0,1,A,5,2,1,1,1,1,1,2,1,1,0,0,0,0,3,0.0,1.0,3,1.0,1.0,2,1,9,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2.10,0,66666.0,240000.0,100000.0,41.0,2,3,1,4,1,8,4,2019,7.0,0,0,30,30,1,3,11001,11,16571555,3,False
1,1,A,4,2,1,1,0,1,1,1,1,1,0,0,0,0,5,0.0,1.0,3,9.0,9.0,1,1,9,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3.60,0,120000.0,156000.0,56000.0,35.0,4,3,1,1,2,8,3,2019,7.0,0,0,30,30,1,2,11001,11,15753361,10,False
2,1,A,4,2,1,1,0,1,1,1,1,1,0,0,0,0,5,0.0,1.0,3,9.0,9.0,9,9,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3.60,0,120000.0,156000.0,56000.0,35.0,4,3,1,1,2,8,3,2019,7.0,0,0,30,30,1,2,11001,11,9899861,14,False
3,1,A,1,2,1,1,1,1,1,1,1,1,1,0,0,0,3,0.0,1.0,0,9.0,9.0,2,1,9,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,4.35,0,30000.0,326666.0,100000.0,30.0,1,2,1,6,1,3,3,2019,7.0,0,0,33,23,0,2,11001,11,14146154,23,False
4,1,A,5,1,1,1,1,1,1,2,1,1,0,0,0,0,4,1.0,0.0,3,9.0,9.0,9,9,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,4.35,0,150000.0,247500.0,75000.0,30.0,2,3,0,6,3,2,2,2019,7.0,0,0,47,31,0,2,11001,11,11313127,33,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990768,3,C,17,1,0,0,0,0,0,1,4,1,0,0,0,0,3,0.0,1.0,0,1.0,1.0,3,1,9,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,5.85,1,345666.0,236666.0,133333.0,56.0,1,2,0,4,4,2,2,2017,0.0,0,0,36,34,1,0,99524,99,11419702,5945432,False
990769,3,C,30,2,1,0,0,0,0,1,2,1,0,0,0,0,4,1.0,1.0,3,9.0,9.0,1,1,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,700000.0,283750.0,70000.0,24.0,1,2,1,4,1,8,4,2019,0.0,1,1,30,30,1,1,99001,99,15322643,5945442,False
990770,3,D,36,1,0,0,0,0,0,2,4,1,0,0,0,0,3,0.0,1.0,1,9.0,9.0,2,1,9,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,2.70,0,266666.0,356666.0,150000.0,42.0,1,2,0,6,4,6,5,2018,0.0,1,6,33,28,1,0,99624,99,17388731,5945444,False
990771,3,D,51,1,1,0,0,0,0,1,3,1,0,0,0,0,3,1.0,1.0,3,9.0,9.0,1,1,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,400000.0,113333.0,100000.0,88.0,1,2,0,6,3,6,5,2019,0.0,0,0,27,23,0,1,99001,99,13495606,5945448,False


# Synthetic Minority Oversampling Technique

You have to install the following library:

**sudo pip install imbalanced-learn**

In [ ]:
# check version number
import imblearn
print(imblearn.__version__)

0.8.0


In [ ]:
# Oversample with SMOTE and random undersample for imbalanced dataset
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from matplotlib import pyplot
from numpy import where

# Preparing the Data

In [ ]:
df_test = df.copy()
# asignar valor temporal de 15 a los NA
df_test[['Ind_fue_atendido_salud', 'Ind_acudio_salud', 'Sexo']] = df_test[['Ind_fue_atendido_salud', 
                                                                           'Ind_acudio_salud', 'Sexo']].fillna(15)

In [ ]:
# cambiar valores de Ind_grupo_sisben_4 a numéricos
dic_ind_grupo_sisben = {'A':0, 'B':1, 'C':2, 'D':3}
df_test['Ind_grupo_sisben_4'] = df_test['Ind_grupo_sisben_4'].map(dic_ind_grupo_sisben)
df_test['Ind_grupo_sisben_4'].unique()

[0, 1, 2, 3]
Categories (4, int64): [0 < 1 < 2 < 3]

In [ ]:
# cambiar variables UInt8
df_test['Ind_nivel_sisben_4']= df_test['Ind_nivel_sisben_4'].astype("int")

# cambiar variables UInt32
df_test[['IdBeneficiario', 'Id']] = df_test[['IdBeneficiario', 'Id']].astype("int")

In [ ]:
# cambiar variable Ind_grupo_sisben_4 a float
df_test['Ind_grupo_sisben_4'] = df_test['Ind_grupo_sisben_4'].astype("int")

In [ ]:
df1 = df_test.drop(columns = ['desnutricion', 'recuperacion', 'union_temp','FechaNacimiento'])
cols_to_X = [col for col in df1.columns if col != 'reincidencia']

In [ ]:
df1

,Cod_clase,Ind_grupo_sisben_4,Ind_nivel_sisben_4,Tip_vivienda,Ind_tiene_energia,Ind_tiene_alcantarillado,Ind_tiene_gas,Ind_tiene_recoleccion,Ind_tiene_acueducto,Num_hogares_vivienda,Tip_ocupa_vivienda,Ind_tiene_cocina,Ind_tiene_nevera,Ind_tiene_lavadora,Ind_tiene_pc,Ind_tiene_internet,Num_personas_hogar,Sexo,Ind_discap_ninguna,Tip_seg_social,Ind_acudio_salud,Ind_fue_atendido_salud,Tip_cuidado_niños,Ind_recibe_comida,Ind_leer_escribir,Ind_estudia,privado_logroeduca,privado_analfabe,privado_Inasistencia,privado_Rezago,privado_primera_inf,privado_trabajo_inf,privado_desemplarga,privado_trabajoinfo,privado_asegura,privado_accesosalud,privado_agua,privado_excreta,privado_suelo,privado_pared,privado_hacina,Noprivaciones,IndicadorPobrezaMulti,ingresos_promP_imp,gasto_ppers_imp,gasto_alim_ppers_imp,porc_gasto_alim,n_ninos,Tip_hijo,Jefat_fem,Tip_familia,Gene_Ingr,Nivel_padreUni,Nivel_madreUni,anno_encuesta,Uni_dias_agua,Ind_afec_eventonatural,num_afec_eventonatural,Edad_padres_mayor,Edad_padres_menor,ComparteCocina_Sanitario,Estrato,cod_mpio,cod_dpto,IdBeneficiario,Id,reincidencia
0,1,0,5,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,3.0,1.0,1.0,2.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.10,0.0,66666.0,240000.0,100000.0,41.0,2.0,3.0,1.0,4.0,1.0,8.0,4.0,2019.0,7.0,0.0,0.0,30.0,30.0,1.0,3.0,11001.0,11.0,16571555,3,False
1,1,0,4,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,3.0,9.0,9.0,1.0,1.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.60,0.0,120000.0,156000.0,56000.0,35.0,4.0,3.0,1.0,1.0,2.0,8.0,3.0,2019.0,7.0,0.0,0.0,30.0,30.0,1.0,2.0,11001.0,11.0,15753361,10,False
2,1,0,4,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,3.0,9.0,9.0,9.0,9.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.60,0.0,120000.0,156000.0,56000.0,35.0,4.0,3.0,1.0,1.0,2.0,8.0,3.0,2019.0,7.0,0.0,0.0,30.0,30.0,1.0,2.0,11001.0,11.0,9899861,14,False
3,1,0,1,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,9.0,9.0,2.0,1.0,9.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.35,0.0,30000.0,326666.0,100000.0,30.0,1.0,2.0,1.0,6.0,1.0,3.0,3.0,2019.0,7.0,0.0,0.0,33.0,23.0,0.0,2.0,11001.0,11.0,14146154,23,False
4,1,0,5,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,3.0,9.0,9.0,9.0,9.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.35,0.0,150000.0,247500.0,75000.0,30.0,2.0,3.0,0.0,6.0,3.0,2.0,2.0,2019.0,7.0,0.0,0.0,47.0,31.0,0.0,2.0,11001.0,11.0,11313127,33,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990768,3,2,17,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,1.0,3.0,1.0,9.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,5.85,1.0,345666.0,236666.0,133333.0,56.0,1.0,2.0,0.0,4.0,4.0,2.0,2.0,2017.0,0.0,0.0,0.0,36.0,34.0,1.0,0.0,99524.0,99.0,11419702,5945432,False
990769,3,2,30,2.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,3.0,9.0,9.0,1.0,1.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,700000.0,283750.0,70000.0,24.0,1.0,2.0,1.0,4.0,1.0,8.0,4.0,2019.0,0.0,1.0,1.0,30.0,30.0,1.0,1.0,99001.0,99.0,15322643,5945442,False
990770,3,3,36,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,1.0,9.0,9.0,2.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.70,0.0,266666.0,356666.0,150000.0,42.0,1.0,2.0,0.0,6.0,4.0,6.0,5.0,2018.0,0.0,1.0,6.0,33.0,28.0,1.0,0.0,99624.0,99.0,17388731,5945444,False
990771,3,3,51,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,3.0,9.0,9.0,1.0,1.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,400000.0,113333.0,100000.0,88.0,1.0,2.0,0.0,6.0,3.0,6.0,5.0,201

In [ ]:
for col in df1.columns[:41]:
  df1[col] = df1[col].astype('category')


In [ ]:
for col in df1.columns[47:57]:
  df1[col] = df1[col].astype('category')

In [ ]:
df1[['Edad_padres_mayor', 'Edad_padres_menor', 'cod_mpio', 'cod_dpto', 'IdBeneficiario', 'Id', 'Estrato', 'anno_encuesta']] = df1[['Edad_padres_mayor', 'Edad_padres_menor', 'cod_mpio', 'cod_dpto', 'IdBeneficiario', 'Id', 'Estrato', 'anno_encuesta']].astype('int')



In [ ]:
df1['ComparteCocina_Sanitario'] = df1['ComparteCocina_Sanitario'].astype('category')

In [ ]:
df1['Noprivaciones'] = df1['Noprivaciones'].astype('category')

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990773 entries, 0 to 990772
Data columns (total 67 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Cod_clase                 990773 non-null  category
 1   Ind_grupo_sisben_4        990773 non-null  category
 2   Ind_nivel_sisben_4        990773 non-null  category
 3   Tip_vivienda              990773 non-null  category
 4   Ind_tiene_energia         990773 non-null  category
 5   Ind_tiene_alcantarillado  990773 non-null  category
 6   Ind_tiene_gas             990773 non-null  category
 7   Ind_tiene_recoleccion     990773 non-null  category
 8   Ind_tiene_acueducto       990773 non-null  category
 9   Num_hogares_vivienda      990773 non-null  category
 10  Tip_ocupa_vivienda        990773 non-null  category
 11  Ind_tiene_cocina          990773 non-null  category
 12  Ind_tiene_nevera          990773 non-null  category
 13  Ind_tiene_lavadora        990

In [ ]:
X = df_test[cols_to_X]
y = df1['reincidencia']
counter = Counter(y)
print(counter)

Counter({False: 988755, True: 2018})


In [ ]:
# transform the dataset with oversample of the minority category
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

[1, 2, 3, NaN]
Categories (3, int64): [1, 2, 3]

In [ ]:
for col in X.columns:
    print(f'{col} has: ')
    print(df[col].isna().value_counts())

Cod_clase has: 
False    990773
Name: Cod_clase, dtype: int64
Ind_grupo_sisben_4 has: 
False    990773
Name: Ind_grupo_sisben_4, dtype: int64
Ind_nivel_sisben_4 has: 
False    990773
Name: Ind_nivel_sisben_4, dtype: int64
Tip_vivienda has: 
False    990773
Name: Tip_vivienda, dtype: int64
Ind_tiene_energia has: 
False    990773
Name: Ind_tiene_energia, dtype: int64
Ind_tiene_alcantarillado has: 
False    990773
Name: Ind_tiene_alcantarillado, dtype: int64
Ind_tiene_gas has: 
False    990773
Name: Ind_tiene_gas, dtype: int64
Ind_tiene_recoleccion has: 
False    990773
Name: Ind_tiene_recoleccion, dtype: int64
Ind_tiene_acueducto has: 
False    990773
Name: Ind_tiene_acueducto, dtype: int64
Num_hogares_vivienda has: 
False    990773
Name: Num_hogares_vivienda, dtype: int64
Tip_ocupa_vivienda has: 
False    990773
Name: Tip_ocupa_vivienda, dtype: int64
Ind_tiene_cocina has: 
False    990773
Name: Ind_tiene_cocina, dtype: int64
Ind_tiene_nevera has: 
False    990773
Name: Ind_tiene_nevera,

In [ ]:
X

,Cod_clase,Ind_grupo_sisben_4,Ind_nivel_sisben_4,Tip_vivienda,Ind_tiene_energia,Ind_tiene_alcantarillado,Ind_tiene_gas,Ind_tiene_recoleccion,Ind_tiene_acueducto,Num_hogares_vivienda,Tip_ocupa_vivienda,Ind_tiene_cocina,Ind_tiene_nevera,Ind_tiene_lavadora,Ind_tiene_pc,Ind_tiene_internet,Num_personas_hogar,Sexo,Ind_discap_ninguna,Tip_seg_social,Ind_acudio_salud,Ind_fue_atendido_salud,Tip_cuidado_niños,Ind_recibe_comida,Ind_leer_escribir,Ind_estudia,privado_logroeduca,privado_analfabe,privado_Inasistencia,privado_Rezago,privado_primera_inf,privado_trabajo_inf,privado_desemplarga,privado_trabajoinfo,privado_asegura,privado_accesosalud,privado_agua,privado_excreta,privado_suelo,privado_pared,privado_hacina,Noprivaciones,IndicadorPobrezaMulti,ingresos_promP_imp,gasto_ppers_imp,gasto_alim_ppers_imp,porc_gasto_alim,n_ninos,Tip_hijo,Jefat_fem,Tip_familia,Gene_Ingr,Nivel_padreUni,Nivel_madreUni,anno_encuesta,Uni_dias_agua,Ind_afec_eventonatural,num_afec_eventonatural,Edad_padres_mayor,Edad_padres_menor,ComparteCocina_Sanitario,Estrato,cod_mpio,cod_dpto,IdBeneficiario,Id
0,1,0.000000,5.000000,2.000000,1.0,1.000000,1.000000,1.000000,1.0,2.000000,1.000000,1.000000,0.000000,0.00000,0.0,0.0,3.000000,0.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.0,9.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.0,1.000000,2.100000,0.000000,66666.000000,240000.000000,100000.000000,41.000000,2.000000,3.000000,1.000000,4.000000,1.000000,8.000000,4.000000,2019.000000,7.000000,0.000000,0.000000,30.000000,30.000000,1.00000,3.0,11001.000000,11.000000,1.657156e+07,3.000000e+00
1,1,0.000000,4.000000,2.000000,1.0,1.000000,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.00000,0.0,0.0,5.000000,0.000000,1.000000,3.000000,9.000000,9.000000,1.000000,1.0,9.0,1.000000,1.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,1.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,3.600000,0.000000,120000.000000,156000.000000,56000.000000,35.000000,4.000000,3.000000,1.000000,1.000000,2.000000,8.000000,3.000000,2019.000000,7.000000,0.000000,0.000000,30.000000,30.000000,1.00000,2.0,11001.000000,11.000000,1.575336e+07,1.000000e+01
2,1,0.000000,4.000000,2.000000,1.0,1.000000,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.00000,0.0,0.0,5.000000,0.000000,1.000000,3.000000,9.000000,9.000000,9.000000,9.0,0.0,1.000000,1.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,1.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,3.600000,0.000000,120000.000000,156000.000000,56000.000000,35.000000,4.000000,3.000000,1.000000,1.000000,2.000000,8.000000,3.000000,2019.000000,7.000000,0.000000,0.000000,30.000000,30.000000,1.00000,2.0,11001.000000,11.000000,9.899861e+06,1.400000e+01
3,1,0.000000,1.000000,2.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.00000,0.0,0.0,3.000000,0.000000,1.000000,0.000000,9.000000,9.000000,2.000000,1.0,9.0,0.000000,1.000000,0.0,0.0,0.0,1.00000,0.0,0.000000,0.000000,1.00000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,4.350000,0.000000,30000.000000,326666.000000,100000.000000,30.000000,1.000000,2.000000,1.000000,6.000000,1.000000,3.000000,3.000000,2019.000000,7.000000,0.000000,0.000000,33.000000,23.000000,0.00000,2.0,11001.000000,11.000000,1.414615e+07,2.300000e+01
4,1,0.000000,5.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,2.000000,1.000000,1.000000,0.000000,0.00000,0.0,0.0,4.000000,1.000000,0.000000,3.000000,9.000000,9.000000,9.000000,9.0,1.0,1.000000,1.000000,0.0,0.0,1.0,0.00000,0.0,0.000000,1.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,4.350000,0.000000,150000.000000,247500.000000,75000.000000,30.000000,2.000000,3.000000,0.000000,6.000000,3.000000,2.000000,2.000000,2019.000000,7.000000,0.000000,0.000000,47.000000,31.000000,0.00000,2.0,11001.000000,11.000000,1.131313e+07,3.300000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
y = y.to_frame(name = 'reincidencia')

In [ ]:
X[X['Ind_grupo_sisben_4'].isna() == True]

,Cod_clase,Ind_grupo_sisben_4,Ind_nivel_sisben_4,Tip_vivienda,Ind_tiene_energia,Ind_tiene_alcantarillado,Ind_tiene_gas,Ind_tiene_recoleccion,Ind_tiene_acueducto,Num_hogares_vivienda,Tip_ocupa_vivienda,Ind_tiene_cocina,Ind_tiene_nevera,Ind_tiene_lavadora,Ind_tiene_pc,Ind_tiene_internet,Num_personas_hogar,Sexo,Ind_discap_ninguna,Tip_seg_social,Ind_acudio_salud,Ind_fue_atendido_salud,Tip_cuidado_niños,Ind_recibe_comida,Ind_leer_escribir,Ind_estudia,privado_logroeduca,privado_analfabe,privado_Inasistencia,privado_Rezago,privado_primera_inf,privado_trabajo_inf,privado_desemplarga,privado_trabajoinfo,privado_asegura,privado_accesosalud,privado_agua,privado_excreta,privado_suelo,privado_pared,privado_hacina,Noprivaciones,IndicadorPobrezaMulti,ingresos_promP_imp,gasto_ppers_imp,gasto_alim_ppers_imp,porc_gasto_alim,n_ninos,Tip_hijo,Jefat_fem,Tip_familia,Gene_Ingr,Nivel_padreUni,Nivel_madreUni,anno_encuesta,Uni_dias_agua,Ind_afec_eventonatural,num_afec_eventonatural,Edad_padres_mayor,Edad_padres_menor,ComparteCocina_Sanitario,Estrato,cod_mpio,cod_dpto,IdBeneficiario,Id
199768,1,NaN,4.517174,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.988551,4.000000,1.000000,0.000000,0.000000,0.000000,0.0,3.494275,1.000000,1.000000,3.000000,9.00000,9.00000,9.000000,9.000000,0.505725,1.000000,0.494275,0.000000,0.0,1.000000,0.000000,0.0,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,5.091413,0.494275,67923.639334,124843.900850,75286.226518,64.874060,2.494275,3.000000,1.000000,4.000000,1.000000,8.000000,3.517174,2019.000000,1.199270,0.505725,1.011449,28.011449,28.011449,1.000000,1.494275,20035.780502,20.000000,1.056543e+07,1.187932e+06
199769,1,NaN,7.287708,2.542458,0.457542,0.457542,0.457542,1.000000,0.457542,1.000000,3.627375,0.457542,1.000000,0.000000,0.000000,0.0,5.627375,0.542458,1.000000,3.000000,9.00000,9.00000,2.830166,1.000000,9.000000,0.542458,0.542458,0.000000,0.0,0.000000,0.457542,0.0,0.000000,1.000000,0.0,0.0,0.542458,0.542458,0.542458,0.542458,0.542458,4.284219,0.542458,90958.845442,87489.221011,63468.288287,78.953088,2.000000,1.915083,0.000000,1.000000,3.000000,3.000000,8.000000,2019.000000,3.202791,0.542458,2.169834,51.575416,51.575416,0.000000,0.915083,24496.163021,23.915083,1.476824e+07,1.394954e+06
199772,2,NaN,12.289348,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.935395,1.000000,0.064605,0.000000,0.000000,0.0,3.064605,0.935395,1.000000,1.129209,9.00000,9.00000,2.000000,1.000000,9.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.064605,0.0,0.0,1.000000,0.064605,1.000000,0.000000,0.064605,2.874433,0.064605,254768.012140,111454.597848,110743.946227,97.997255,1.129209,2.064605,0.064605,5.870791,2.935395,4.258419,3.000000,2018.935395,0.000000,0.000000,0.000000,32.000000,26.387628,1.000000,0.064605,23160.713184,22.741581,1.688627e+07,4.204379e+06
199775,1,NaN,3.848445,2.075778,1.000000,0.462111,0.462111,1.000000,1.000000,1.462111,4.000000,0.462111,0.000000,0.000000,0.000000,0.0,6.227333,0.000000,1.000000,3.000000,9.00000,9.00000,1.537889,1.000000,9.000000,0.462111,0.537889,0.000000,0.0,0.000000,0.000000,0.0,0.462111,0.537889,0.0,0.0,0.000000,0.537889,0.537889,0.537889,1.000000,3.875033,0.537889,79082.042731,75113.648702,54713.035243,72.689444,4.227333,2.537889,0.537889,6.000000,2.613666,3.386334,3.848445,2019.000000,0.217805,0.462111,2.310556,33.303110,31.303110,1.000000,1.000000,44042.127897,44.000000,1.431695e+07,1.837473e+06
199776,1,NaN,13.910354,1.782071,1.000000,1.000000,1.000000,1.000000,1.000000,1.435858,1.435858,1.000000,0.782071,0.782071,0.000000,0.0,3.564142,0.000000,1.000000,1.435858,9.00000,9.00000,2.743433,2.743433,7.256567,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.217929,0.000000,0.782071,3.600000,0.000000,351844.260118,250690.772683,95641.416983,39.794441,1.782071,1.217929,0.217929,5.564142,3.564142,4.871717,2.000000,2018.435858,0.871717,0.000000,0.000000,26.000000,22.871717,0.217929,2.564142,

In [ ]:
y

,reincidencia
0,False
1,False
2,False
3,False
4,False
...,...
296620,True
296621,True
296622,True
296623,True


In [ ]:
# train test split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)


print(f"""X_train shape {X_train.shape}
y_train shape {y_train.shape}
X_test shape {X_test.shape}
y_test shape {y_test.shape}""")



X_train shape (207637, 66)
y_train shape (207637, 1)
X_test shape (88988, 66)
y_test shape (88988, 1)


In [ ]:
parameters = {'max_depth':range(3,20), 'min_samples_leaf':range(1,5), 'n_estimators':[50,150]}
classifierRF1 = GridSearchCV(RandomForestClassifier(criterion='gini', random_state=202), parameters, n_jobs=6, cv=5, verbose=1) #5 kfolds
classifierRF1.fit(X=X_train, y=y_train['reincidencia'])
clfRF_model1 = classifierRF1.best_estimator_
print('Best classifierRF ', classifierRF1.best_score_, classifierRF1.best_params_)

Fitting 5 folds for each of 136 candidates, totalling 680 fits


/Users/giovannicastellanosuribe/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').